## 应用简化版SMO算法处理小规模数据集

#### SMO函数伪代码
     创建一个alpha向量并将其初始化为0向量
     当迭代次数小于最大迭代次数时（外循环）:
     对数据集中的每个数据向量（内循环）:
     如果该数据向量可以被优化：
         随机选择另外一个数据向量
         同时优化这两个向量
         如果两个向量都不能被优化，退出内循环
     如果所有向量都没有被优化，增加迭代数目，继续下一次循环
    
### 5-1 SMO算法中的辅助函数

In [1]:
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

def selectJrand(i,m):
    j=1
    while(j == i):
        j = int(random.uniform(0,m))
    return j

def clipAlpha(aj,H,L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [2]:
dataArr, labelArr = loadDataSet('testSet.txt')
print(labelArr)

[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


### 5-2 简化版SMO算法

In [3]:
from numpy import *
def smoSimple(dataMatIn, classLabels, C, toler,maxIter):
    dataMatrix = mat(dataMatIn)   # 把特征集合转化成矩阵形式
    labelMat = mat(classLabels).transpose()   # 把类别集合转化成矩阵形式（一维多列）/（多维一列）， 然后在 转置（多维一列）/ （一维多列）
    b = 0   # 初始化参数b 为 0
    m,n = shape(dataMatrix)  # 取特征矩阵的行（m）， 列（n）
    alphas = mat(zeros((m,1)))   # 先建立一个m行1列 的数组初始化为全0， 然后把多维数组 转化为 矩阵   长的很像还不是一回事， 但是可以用  == 
    iter = 0  # 初始化迭代次数 为 0
    while(iter < maxIter):  # 没有到最大迭代次数之前一直执行while语句
        alphaParisChanged = 0   # 记录alpha是否已经进行优化，并每次循环时设为 把 alpha 初始化 为 0
        for i in range(m):   # 遍历整个集合
            fXi = float(multiply(alphas, labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b  # multiply函数，对应位置相乘，生成矩阵后转置
            # 乘以  特征矩阵第i行转置在 后 * 整个矩阵   （  sum(alpha_i * y_i * x_i.T * x) + b  x 指的是特征矩阵， 这里是dataMatrix）
            Ei = fXi - float(labelMat[i])  # 误差 = 预测的类别 - 实际类别  （第一个 alpha 的误差）-- alpha[i]
            if ((labelMat[i] * Ei < -toler) and (alphas[i] < C)) or (labelMat[i]*Ei > toler) and (alphas[i] > 0):  # 正负间隔都要测试，
                # 同时检查alpha， 若alpha不等于0 或 C， 对alpha进行优化
                j = selectJrand(i,m)   # 借助辅助函数， 随机选取第二个 alpha的值  （alpha[j]）
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b  # 计算同上
                Ej = fXj - float(labelMat[j])  # 误差 = 预测的类别 - 实际类别  （第二个alpha的误差）-- alpha[j]
                alphaIold = alphas[i].copy()  # 返回给定对象的数组副本   这里是  第i个alpha 取它的副本   下次循环就变成  old值
                alphaJold = alphas[j].copy()  # 同上
                
                
                if labelMat[i] != labelMat[j]:  # 如果第 i 个 分类标签 和 第 j 个 分类标签不一样
                    L = max(0, alphas[j] - alphas[i])  # 用于调整alpha到 0  到  C 之间
                    H = min(C, C + alphas[j] - alphas[i])
                else:   # 如果第 i 个 分类标签 和 第 j 个 分类标签不一样
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, C + alphas[j] + alphas[i] )
                if L == H:
                    print("L==H")
                    continue  # 若 L==H 本次循环结束，直接运行下一次 for 循环
                    
                    
                eta = 2.0*dataMatrix[i,:]*dataMatrix[j,:].T-dataMatrix[i,:]*dataMatrix[i,:].T -dataMatrix[j,:] * dataMatrix[j,:].T #alpha[j]
                # 的最优修正量
                if eta >= 0:
                    print("eta >= 0")
                    continue  # 若eta >= 0, 本次循环结束，直接运行下一次 for 循环
                    
                    
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)  # 调用clipAlpha函数， 改变alpha[j]
                if (abs(alphas[j] - alphaJold) < 0.00001):
                    print("j not moving enough")
                    continue   # 若alpha[j] 改变的程度太小，本次循环结束，直接运行下一次 for 循环
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])  # 对alpha[j]进行修改， 修改量与j相同， 但方向相反
                
                # 设置常数项
                temp1 = labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[i,:]*dataMatrix[j,:].T 
                b1 = b - Ei - labelMat[i]*(alphas[i] - alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - temp1 
                     #labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[i,:]*dataMatrix[j,:].T  
                temp2 = labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[j,:]*dataMatrix[j,:].T
                b2 = b - Ej - labelMat[i]*(alphas[i] - alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - temp2  
                     #labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[j,:]*dataMatrix[j,:].T
                if 0 < alphas[i] and C > alphas[i]:   # alpha[i] 不在约束条件内    [0, C](这是数学上的集合)
                    b = b1   # b 设置 为 b1
                elif 0 < alphas[j] and C > alphas[j]:  # alpha[j] 在约束条件内   [0,C]
                    b = b2  # b 设置 为 b2
                else:  # 其他情况
                    b = (b1 + b2)/2.0  # b 设置为 其平均数
                alphaParisChanged += 1  #执行到此行， 说明已经成功的改变了 一对 alpha， 增加 alphaPairsChanged
                print("iter: %d i:%d, pairs changed %d" %(iter, i, alphaParisChanged))
            if alphaParisChanged == 0:  # 如果没有成功的改变了 一对 alpha， 迭代次数加1
                iter += 1
            else:  # 否则， 迭代次数设置为0
                iter = 0
            print("iteration number: %d" % iter)  # 输出迭代次数
        return b,alphas  

In [4]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)
print("b =",b)
print("alphas =", alphas[alphas > 0])

j not moving enough
j not moving enough
iter: 0 i:2, pairs changed 1
iteration number: 0
iter: 0 i:3, pairs changed 2
iteration number: 0
iter: 0 i:4, pairs changed 3
iteration number: 0
iter: 0 i:5, pairs changed 4
iteration number: 0
iteration number: 0
iter: 0 i:7, pairs changed 5
iteration number: 0
iter: 0 i:8, pairs changed 6
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
iteration number: 0
iteration number: 0
j not moving enough
j not moving enough
iteration number: 0
iter: 0 i:25, pairs changed 7
iteration number: 0
iter: 0 i:26, pairs changed 8
iteration number: 0
iter: 0 i:27, pairs changed 9
iteration number: 0
iteration number: 0
j not moving enough
j not moving enough
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number

### 5-3 完整版Platt SMO的支持函数

In [5]:
class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2)))
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)
        
def calcEk(oS, k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek
        
def selectJ(i, oS, Ei):
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:
            if k == i: continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1,Ek]

### 5-4 完整Platt SMO算法中的优化例程

In [20]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H:
            print("L==H")
            return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j]
        if eta >= 0:
            print("eta>=0")
            return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            print("j not moving enough")
            return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])
        updateEk(oS, i)
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else:
            oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

### 5-5 完整版Platt SMO的外循环代码

In [21]:
def smoP(dataMatIn, classLabels, C, toler, maxIter,kTup=('lin', 0)):
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:   #go over all
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i,oS)
                print("fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        else:#go over non-bound (railed) alphas
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet: entireSet = False #toggle entire set loop
        elif (alphaPairsChanged == 0): entireSet = True  
        print("iteration number: %d" % iter)
    return oS.b,oS.alphas

### 5-6核转化函数

In [12]:
def kernelTrans(X, A, kTup):
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0] == 'lin':
        K = X * A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow * deltaRow.T
        K = exp(K /(-1*kTup[1]**2))
    else:
        raise NameError('Houston We Have a Problem -- That kernel is not recognized')
    return K   
        
class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m, 1)))
        self.b = 0
        self.eCache = mat(zeros((self.m, 2)))
        self.K = mat(zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)

### 5-7 使用核函数需要对innerL（）及calcEk（）函数进行的修改

In [19]:
def innerLK(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H:
            print("L==H")
            return 0
        eta = 2.0 * oS.X[i,:]*oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T - oS.X[j,:]*oS.X[j,:].T
        if eta >= 0:
            print("eta>=0")
            return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print("j not moving enough"); return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])
        updateEk(oS, i)
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[i,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[i,:]*oS.X[j,:].T
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[j,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[j,:]*oS.X[j,:].T
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

### 5-9 基于SVM的手写熟悉识别

In [27]:
from numpy import *
def img2vector(filename):
    returnVect = zeros((1,1024))  # 将32x32的图像转化为1x1024的向量

    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9:
            hwLabels.append(-1)
        else:
            hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels

def testDigits(kTup=('rbf',10)):
    dataArr, labelArr = loadImages('trainingDigits')
    b,alphas = smoP(dataArr,labelArr,200,0.0001,1000,kTup)
    dataMat=mat(dataArr)
    labelMat = mat(labelArr).transpose()
    svInd =nonzero(alphas.A>0)[0]
    sVs = dataMat[svInd]
    labelSV = labelMat[svInd]
    print("there are %d Suppoer Vector "% shape(sVs)[0])
    m,n = shape(dataMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,dataMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print("the training eror rate is:%f"%(float(errorCount)/m))
    dataArr,labelArr = loadImages('testDigits')
    errorCount = 0
    dataMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    m,n = shape(dataMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,dataMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print("the test eror rate is:%f"%(float(errorCount)/m))

In [28]:
testDigits(('rbf',20))

L==H
fullSet, iter: 0 i:0, pairs changed 0
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 4
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 4
fullSet, iter: 0 i:7, pairs changed 5
fullSet, iter: 0 i:8, pairs changed 6
fullSet, iter: 0 i:9, pairs changed 7
fullSet, iter: 0 i:10, pairs changed 7
fullSet, iter: 0 i:11, pairs changed 7
fullSet, iter: 0 i:12, pairs changed 7
fullSet, iter: 0 i:13, pairs changed 8
fullSet, iter: 0 i:14, pairs changed 9
fullSet, iter: 0 i:15, pairs changed 9
fullSet, iter: 0 i:16, pairs changed 9
fullSet, iter: 0 i:17, pairs changed 10
fullSet, iter: 0 i:18, pairs changed 11
fullSet, iter: 0 i:19, pairs changed 11
j not moving enough
fullSet, iter: 0 i:20, pairs changed 11
L==H
fullSet, iter: 0 i:21, pairs changed 11
L==H
fullSet, iter: 0 i:22, pairs changed 11
fullSet, iter: 0 i:23, pairs changed 11
fullSet, iter: 0 i:24, pairs cha

L==H
fullSet, iter: 0 i:197, pairs changed 11
L==H
fullSet, iter: 0 i:198, pairs changed 11
L==H
fullSet, iter: 0 i:199, pairs changed 11
L==H
fullSet, iter: 0 i:200, pairs changed 11
L==H
fullSet, iter: 0 i:201, pairs changed 11
L==H
fullSet, iter: 0 i:202, pairs changed 11
L==H
fullSet, iter: 0 i:203, pairs changed 11
L==H
fullSet, iter: 0 i:204, pairs changed 11
L==H
fullSet, iter: 0 i:205, pairs changed 11
L==H
fullSet, iter: 0 i:206, pairs changed 11
L==H
fullSet, iter: 0 i:207, pairs changed 11
L==H
fullSet, iter: 0 i:208, pairs changed 11
L==H
fullSet, iter: 0 i:209, pairs changed 11
L==H
fullSet, iter: 0 i:210, pairs changed 11
L==H
fullSet, iter: 0 i:211, pairs changed 11
L==H
fullSet, iter: 0 i:212, pairs changed 11
L==H
fullSet, iter: 0 i:213, pairs changed 11
L==H
fullSet, iter: 0 i:214, pairs changed 11
L==H
fullSet, iter: 0 i:215, pairs changed 11
L==H
fullSet, iter: 0 i:216, pairs changed 11
L==H
fullSet, iter: 0 i:217, pairs changed 11
L==H
fullSet, iter: 0 i:218, pairs

L==H
fullSet, iter: 0 i:385, pairs changed 11
L==H
fullSet, iter: 0 i:386, pairs changed 11
L==H
fullSet, iter: 0 i:387, pairs changed 11
L==H
fullSet, iter: 0 i:388, pairs changed 11
L==H
fullSet, iter: 0 i:389, pairs changed 11
L==H
fullSet, iter: 0 i:390, pairs changed 11
L==H
fullSet, iter: 0 i:391, pairs changed 11
L==H
fullSet, iter: 0 i:392, pairs changed 11
L==H
fullSet, iter: 0 i:393, pairs changed 11
L==H
fullSet, iter: 0 i:394, pairs changed 11
L==H
fullSet, iter: 0 i:395, pairs changed 11
L==H
fullSet, iter: 0 i:396, pairs changed 11
L==H
fullSet, iter: 0 i:397, pairs changed 11
L==H
fullSet, iter: 0 i:398, pairs changed 11
L==H
fullSet, iter: 0 i:399, pairs changed 11
L==H
fullSet, iter: 0 i:400, pairs changed 11
L==H
fullSet, iter: 0 i:401, pairs changed 11
L==H
fullSet, iter: 0 i:402, pairs changed 11
L==H
fullSet, iter: 0 i:403, pairs changed 11
L==H
fullSet, iter: 0 i:404, pairs changed 11
L==H
fullSet, iter: 0 i:405, pairs changed 11
L==H
fullSet, iter: 0 i:406, pairs

L==H
fullSet, iter: 0 i:566, pairs changed 11
L==H
fullSet, iter: 0 i:567, pairs changed 11
L==H
fullSet, iter: 0 i:568, pairs changed 11
L==H
fullSet, iter: 0 i:569, pairs changed 11
L==H
fullSet, iter: 0 i:570, pairs changed 11
L==H
fullSet, iter: 0 i:571, pairs changed 11
L==H
fullSet, iter: 0 i:572, pairs changed 11
L==H
fullSet, iter: 0 i:573, pairs changed 11
L==H
fullSet, iter: 0 i:574, pairs changed 11
L==H
fullSet, iter: 0 i:575, pairs changed 11
L==H
fullSet, iter: 0 i:576, pairs changed 11
L==H
fullSet, iter: 0 i:577, pairs changed 11
L==H
fullSet, iter: 0 i:578, pairs changed 11
L==H
fullSet, iter: 0 i:579, pairs changed 11
L==H
fullSet, iter: 0 i:580, pairs changed 11
L==H
fullSet, iter: 0 i:581, pairs changed 11
L==H
fullSet, iter: 0 i:582, pairs changed 11
L==H
fullSet, iter: 0 i:583, pairs changed 11
L==H
fullSet, iter: 0 i:584, pairs changed 11
L==H
fullSet, iter: 0 i:585, pairs changed 11
L==H
fullSet, iter: 0 i:586, pairs changed 11
L==H
fullSet, iter: 0 i:587, pairs

L==H
fullSet, iter: 0 i:745, pairs changed 11
L==H
fullSet, iter: 0 i:746, pairs changed 11
L==H
fullSet, iter: 0 i:747, pairs changed 11
L==H
fullSet, iter: 0 i:748, pairs changed 11
L==H
fullSet, iter: 0 i:749, pairs changed 11
L==H
fullSet, iter: 0 i:750, pairs changed 11
L==H
fullSet, iter: 0 i:751, pairs changed 11
L==H
fullSet, iter: 0 i:752, pairs changed 11
L==H
fullSet, iter: 0 i:753, pairs changed 11
L==H
fullSet, iter: 0 i:754, pairs changed 11
L==H
fullSet, iter: 0 i:755, pairs changed 11
L==H
fullSet, iter: 0 i:756, pairs changed 11
L==H
fullSet, iter: 0 i:757, pairs changed 11
L==H
fullSet, iter: 0 i:758, pairs changed 11
L==H
fullSet, iter: 0 i:759, pairs changed 11
L==H
fullSet, iter: 0 i:760, pairs changed 11
L==H
fullSet, iter: 0 i:761, pairs changed 11
L==H
fullSet, iter: 0 i:762, pairs changed 11
L==H
fullSet, iter: 0 i:763, pairs changed 11
L==H
fullSet, iter: 0 i:764, pairs changed 11
L==H
fullSet, iter: 0 i:765, pairs changed 11
L==H
fullSet, iter: 0 i:766, pairs

...

L==H
fullSet, iter: 7 i:1129, pairs changed 0
L==H
fullSet, iter: 7 i:1130, pairs changed 0
L==H
fullSet, iter: 7 i:1131, pairs changed 0
L==H
fullSet, iter: 7 i:1132, pairs changed 0
L==H
fullSet, iter: 7 i:1133, pairs changed 0
L==H
fullSet, iter: 7 i:1134, pairs changed 0
L==H
fullSet, iter: 7 i:1135, pairs changed 0
L==H
fullSet, iter: 7 i:1136, pairs changed 0
fullSet, iter: 7 i:1137, pairs changed 0
fullSet, iter: 7 i:1138, pairs changed 0
fullSet, iter: 7 i:1139, pairs changed 0
fullSet, iter: 7 i:1140, pairs changed 0
fullSet, iter: 7 i:1141, pairs changed 0
fullSet, iter: 7 i:1142, pairs changed 0
L==H
fullSet, iter: 7 i:1143, pairs changed 0
fullSet, iter: 7 i:1144, pairs changed 0
fullSet, iter: 7 i:1145, pairs changed 0
fullSet, iter: 7 i:1146, pairs changed 0
fullSet, iter: 7 i:1147, pairs changed 0
fullSet, iter: 7 i:1148, pairs changed 0
L==H
fullSet, iter: 7 i:1149, pairs changed 0
L==H
fullSet, iter: 7 i:1150, pairs changed 0
fullSet, iter: 7 i:1151, pairs changed 0
fu

L==H
fullSet, iter: 7 i:1354, pairs changed 0
fullSet, iter: 7 i:1355, pairs changed 0
fullSet, iter: 7 i:1356, pairs changed 0
fullSet, iter: 7 i:1357, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1358, pairs changed 0
L==H
fullSet, iter: 7 i:1359, pairs changed 0
fullSet, iter: 7 i:1360, pairs changed 0
L==H
fullSet, iter: 7 i:1361, pairs changed 0
fullSet, iter: 7 i:1362, pairs changed 0
fullSet, iter: 7 i:1363, pairs changed 0
fullSet, iter: 7 i:1364, pairs changed 0
fullSet, iter: 7 i:1365, pairs changed 0
fullSet, iter: 7 i:1366, pairs changed 0
L==H
fullSet, iter: 7 i:1367, pairs changed 0
fullSet, iter: 7 i:1368, pairs changed 0
fullSet, iter: 7 i:1369, pairs changed 0
L==H
fullSet, iter: 7 i:1370, pairs changed 0
fullSet, iter: 7 i:1371, pairs changed 0
L==H
fullSet, iter: 7 i:1372, pairs changed 0
fullSet, iter: 7 i:1373, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1374, pairs changed 0
fullSet, iter: 7 i:1375, pairs changed 0
fullSet, iter: 7 i:1376, pai

L==H
fullSet, iter: 7 i:1549, pairs changed 0
fullSet, iter: 7 i:1550, pairs changed 0
L==H
fullSet, iter: 7 i:1551, pairs changed 0
fullSet, iter: 7 i:1552, pairs changed 0
fullSet, iter: 7 i:1553, pairs changed 0
fullSet, iter: 7 i:1554, pairs changed 0
L==H
fullSet, iter: 7 i:1555, pairs changed 0
fullSet, iter: 7 i:1556, pairs changed 0
fullSet, iter: 7 i:1557, pairs changed 0
fullSet, iter: 7 i:1558, pairs changed 0
fullSet, iter: 7 i:1559, pairs changed 0
fullSet, iter: 7 i:1560, pairs changed 0
fullSet, iter: 7 i:1561, pairs changed 0
fullSet, iter: 7 i:1562, pairs changed 0
fullSet, iter: 7 i:1563, pairs changed 0
fullSet, iter: 7 i:1564, pairs changed 0
L==H
fullSet, iter: 7 i:1565, pairs changed 0
fullSet, iter: 7 i:1566, pairs changed 0
fullSet, iter: 7 i:1567, pairs changed 0
fullSet, iter: 7 i:1568, pairs changed 0
L==H
fullSet, iter: 7 i:1569, pairs changed 0
L==H
fullSet, iter: 7 i:1570, pairs changed 0
fullSet, iter: 7 i:1571, pairs changed 0
fullSet, iter: 7 i:1572, pa

j not moving enough
fullSet, iter: 7 i:1736, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1737, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1738, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1739, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1740, pairs changed 0
fullSet, iter: 7 i:1741, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1742, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1743, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1744, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1745, pairs changed 0
fullSet, iter: 7 i:1746, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1747, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1748, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1749, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1750, pairs changed 0
fullSet, iter: 7 i:1751, pairs changed 0
fullSet, iter: 7 i:1752, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1753

j not moving enough
fullSet, iter: 7 i:1889, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1890, pairs changed 0
fullSet, iter: 7 i:1891, pairs changed 0
fullSet, iter: 7 i:1892, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1893, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1894, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1895, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1896, pairs changed 0
fullSet, iter: 7 i:1897, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1898, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1899, pairs changed 0
fullSet, iter: 7 i:1900, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1901, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1902, pairs changed 0
fullSet, iter: 7 i:1903, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1904, pairs changed 0
fullSet, iter: 7 i:1905, pairs changed 0
fullSet, iter: 7 i:1906, pairs changed 0
j not moving enough
fullSet, iter: 7 i:190